In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.ops.rnn_cell import BasicRNNCell,BasicLSTMCell,GRUCell,DropoutWrapper,MultiRNNCell
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

print ("MNIST loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

print ("parameters ready")

parameters ready


In [4]:
#def RNN(x, weights, biases):
with tf.variable_scope("model"):
    #tf.get_variable_scope().reuse_variables()

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x_t = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x_t = tf.reshape(x_t, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x_t = tf.split(0, n_steps, x_t)

    # Define a lstm cell with tensorflow
    cell = rnn_cell.BasicRNNCell(n_hidden)
    
    num_layers=3
    cell = DropoutWrapper(cell, output_keep_prob=0.5)
    cell = MultiRNNCell([cell] * num_layers)    
    
    # Get lstm cell output
    #outputs, states = rnn.rnn(lstm_cell, x_t, dtype=tf.float32)
    state = cell.zero_state(batch_size, tf.float32)
    outputs=[]
    for i, x_ in enumerate(x_t): 
        if i > 0 :
            tf.get_variable_scope().reuse_variables()
        (hidden, state) = cell(x_,state)
        outputs.append(hidden)

    # Linear activation, using rnn inner loop last output
    pred = tf.matmul(outputs[-1], weights['out']) + biases['out']

#pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



print ("Network ready")

Network ready


In [4]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))

        # Run optimization op (backprop)
        _, acc, loss = sess.run([optimizer,accuracy,cost], feed_dict={x: batch_x, y: batch_y})
        
        if step % display_step == 0:
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter 1280, Minibatch Loss= 1.883030, Training Accuracy= 0.32031
Iter 2560, Minibatch Loss= 1.370798, Training Accuracy= 0.57031
Iter 3840, Minibatch Loss= 1.308891, Training Accuracy= 0.59375
Iter 5120, Minibatch Loss= 1.013508, Training Accuracy= 0.69531
Iter 6400, Minibatch Loss= 0.745936, Training Accuracy= 0.77344
Iter 7680, Minibatch Loss= 1.192046, Training Accuracy= 0.66406
Iter 8960, Minibatch Loss= 0.583129, Training Accuracy= 0.78125
Iter 10240, Minibatch Loss= 0.699750, Training Accuracy= 0.72656
Iter 11520, Minibatch Loss= 0.516218, Training Accuracy= 0.85938
Iter 12800, Minibatch Loss= 0.642916, Training Accuracy= 0.78906
Iter 14080, Minibatch Loss= 0.464755, Training Accuracy= 0.85156
Iter 15360, Minibatch Loss= 0.395039, Training Accuracy= 0.89062
Iter 16640, Minibatch Loss= 0.520794, Training Accuracy= 0.83594
Iter 17920, Minibatch Loss= 0.609092, Training Accuracy= 0.78125
Iter 19200, Minibatch Loss= 0.367607, Training Accuracy= 0.88281
Iter 20480, Minibatch Loss= 0.29